In [1]:
import onnxruntime as ort
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write

def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)

    text_norm = torch.LongTensor(text_norm)
    return text_norm

providers = ["CPUExecutionProvider"]
sess_options = ort.SessionOptions()
onnx_sess = ort.InferenceSession(
    "onnx/vits2.onnx",
    sess_options=sess_options,
    providers=providers,
)

In [2]:
import numpy as np

hps = utils.get_hparams_from_file("./configs/vits2_ljs_nosdp.json")
stn_tst = get_text("Hello", hps)
phoneme_ids = stn_tst
text = np.expand_dims(np.array(phoneme_ids, dtype=np.int64), 0)
text_lengths = np.array([text.shape[1]], dtype=np.int64)
scales = np.array([0.667, 1.0, 0.8], dtype=np.float32)
sid=None

In [3]:
text.shape, text_lengths

((1, 6), array([6]))

In [4]:
audio, attention = onnx_sess.run(˜
    None,
    {
        "input": text,
        "input_lengths": text_lengths,
        "scales": scales,
        "sid": sid  
    },
)

In [7]:
import IPython.display as ipd
import soundfile as sf

ipd.Audio(audio[0, 0], rate=22050)
sf.write("hello.wav", audio[0, 0], 22050)